# Demo

In [8]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
from model import bbox_3D_net
from preprocess_data import get_cam_data, get_dect2D_data
from post_processing import gen_3D_box,draw_3D_box,draw_2D_box

### Load 3D box estimator with saved weights

In [2]:
model = bbox_3D_net((224,224,3))

model.load_weights('weights_final.h5')

Instructions for updating:
dim is deprecated, use axis instead


### Path of validation dataset

In [5]:
image_dir = 'test/image_2/'
box2d_dir = 'test/label_2/'
calib_dir = 'test/calib/'

In [4]:
# For each class get average dimensions as per VOC data
classes = ['Car','Van','Truck','Pedestrian','Person_sitting','Cyclist','Tram']
cls_to_ind = {cls:i for i,cls in enumerate(classes)}

dims_avg = np.loadtxt(r'voc_dims.txt',delimiter=',')

# get validation images
all_image = sorted(os.listdir(image_dir))

### Estimating 3D bounding boxes

In [ ]:
# Iterate over all the images
for f in all_image:
    # load label and camera calibration files for corresponding image
    image_file = image_dir + f
    box2d_file = box2d_dir + f.replace('png', 'txt')
    calib_file = calib_dir + f.replace('png', 'txt')
    cam_to_img = get_cam_data(calib_file)
    fx = cam_to_img[0][0]
    u0 = cam_to_img[0][2]
    v0 = cam_to_img[1][2]
    
    # directory to store labels of predicted boxes for evaluation purposes
    box3d_file = 'output_label/'+ f.replace('png','txt')
    
    with open(box3d_file, 'w') as box3d:
        img = cv2.imread(image_file)
        
        # iterate over objects
        for line in open(box2d_file):
            line = line.strip().split(' ')
            cls = line[0]
            
            if cls not in classes:
                # if class of current object is not in our list of class skip it
                box3d.write(' '.join([str(item) for item in line])+'\n')
            
            if cls in classes:
                # get the truncated and occluded state of object
                truncated = np.abs(float(line[1]))
                occluded  = np.abs(float(line[2]))

                # get 2D box coordinates
                obj = {'xmin':int(float(line[4])),
                       'ymin':int(float(line[5])),
                       'xmax':int(float(line[6])),
                       'ymax':int(float(line[7])),
                      }
                
                box_2D = np.array([float(number) for number in line[4:8]])
                
                # performing predictions on easy split(with less truncation and occlusion)
                if truncated < 0.3 and occluded <= 1: 
                    # generate patch to estimate bounding box
                    patch = img[obj['ymin']:obj['ymax'],obj['xmin']:obj['xmax']]
                    patch = cv2.resize(patch, (224, 224))
                    patch = patch - np.array([[[103.939, 116.779, 123.68]]])
                    patch = np.expand_dims(patch, 0)
                    
                    # predict dimension and orientation of object in the patch
                    prediction = model.predict(patch)

                    # compute dims
                    dims = dims_avg[cls_to_ind[cls]] + prediction[0][0]
                    
                    # for bin with maximum confidence get the theta localized
                    max_anc = np.argmax(prediction[2][0])
                    anchors = prediction[1][0][max_anc]
                    if anchors[1] > 0:
                        angle_offset = np.arccos(anchors[0])
                    else:
                        angle_offset = -np.arccos(anchors[0])
                    
                    # calculate theta loc in image coordinates
                    bin_num = prediction[2][0].shape[0]
                    wedge = 2. * np.pi / bin_num
                    theta_loc = angle_offset + max_anc * wedge

                    if theta_loc > 2*np.pi: 
                        theta_l = theta_loc % (2*np.pi)
                    else:
                        theta_l = theta_loc
                    theta_loc = theta_l - np.pi / 2

                    if theta_loc > np.pi:
                        theta_loc -= 2 * np.pi
                    theta_loc = round(theta_loc, 2)

                    # get the observers angle
                    box2d_center_x = (obj['xmin'] + obj['xmax']) / 2.0
                    u_dist = box2d_center_x - u0
                    
                    # Transfer arctan() from (-pi/2,pi/2) to (0,pi)
                    theta_ray = np.arctan(fx / u_dist)
                    
                    if theta_ray<0:
                        theta_ray = theta_ray+np.pi

                    rot_global = theta_loc + theta_ray
                    rot_global = round(rot_global, 2)
                    
                    # calculate yaw of object
                    yaw = - rot_global
                    
                    # store alpha in results
                    line[3] = str(theta_loc)
                    
                    yrot = yaw - np.pi/2
                    if yrot < -np.pi:
                        yrot = yrot + 2.*np.pi
                    
                    # store rotation-y
                    line[-1] = str(yrot)

                    line = line[:8] + list(dims) + line[11:]

                    line = ' '.join([str(item) for item in line]) + '\n'
                    box3d.write(line)
                    
                    # generate and draw 3D box
                    points2D, error = gen_3D_box(yaw, dims, cam_to_img, box_2D)
                    draw_3D_box(img, points2D)
                else:
                    # if object is too much truncated or occluded draw 2D box
                    box3d.write(' '.join([str(item) for item in line])+'\n')
                    draw_2D_box(img,box_2D)
    
    # uncomment following 3 lines to see each of the estimated box drawn on image or use next cell
    # cv2.imshow(f, img)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    cv2.imwrite('output/'+ f.replace('png','jpg'), img)

In [24]:
result_dir = 'output/'
all_results = sorted(os.listdir(result_dir))

# number results to check
num_show = 5
for i in range(num_show):
    idx = np.random.choice(all_results)
    img = cv2.imread(result_dir+idx)
    cv2.imshow('Result',img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Evaluation Code
--label_path = gt_label_directory<br/>
--result_path = predicted_labels_directory<br/>
--label_split_file = file with list of images used for testing<br/>
--current_class = class to evaluate<br/>
--coco = if dataset in coco format<br/>
<br/>
## Usage:
python evaluate.py evaluate --label_path=test/label_2/ --result_path=output_label/ --label_split_file=test/val.txt  --current_class=0 --coco=False

In [28]:
!python kitti-object-eval-python-master/evaluate.py evaluate --label_path=test/label_2/ --result_path=output_label/ --label_split_file=test/val.txt  --current_class=0 --coco=False

C:\Users\Sumit\anaconda3\envs\myenv2\lib\site-packages\numba\cuda\compiler.py:726: NumbaPerformanceWarning: Grid size (1) < 2 * SM count (10) will likely result in GPU under utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
C:\Users\Sumit\anaconda3\envs\myenv2\lib\site-packages\numba\core\typed_passes.py:329: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see https://numba.readthedocs.io/en/stable/user/parallel.html#diagnostics for help.

File "kitti-object-eval-python-master\eval.py", line 135:
@numba.jit(nopython=True, parallel=True)
def d3_box_overlap_kernel(boxes,
^

  warnings.warn(errors.NumbaPerformanceWarning(msg,
C:\Users\Sumit\anaconda3\envs\myenv2\lib\site-packages\numba\cuda\compiler.py:726: NumbaPerformanceWarning: Grid size (1) < 2 * SM count (10) will likely result in GPU under utilization due to low occupanc

Car AP(Average Precision)@0.70, 0.70, 0.70:
bbox AP:100.00, 100.00, 100.00
bev  AP:44.28, 57.90, 46.38
3d   AP:25.61, 36.20, 27.21
aos  AP:87.93, 82.79, 87.01
Car AP(Average Precision)@0.70, 0.50, 0.50:
bbox AP:100.00, 100.00, 100.00
bev  AP:50.99, 67.44, 49.10
3d   AP:50.66, 60.81, 48.70
aos  AP:87.93, 82.79, 87.01

